In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('../ssd_data/imputated_data/imputated_data_MB1.csv')

data

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
0,2019-01-01,31346.0,100.0,0.0,97.0,12769.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.003245e+10,99.0,2.761588e+10,0.0
1,2019-01-02,31346.0,100.0,0.0,97.0,12792.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.058151e+10,99.0,2.778027e+10,0.0
2,2019-01-03,31346.0,100.0,0.0,97.0,12816.0,99.0,13.0,99.0,31.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.106840e+10,99.0,2.795260e+10,0.0
3,2019-01-04,31346.0,100.0,0.0,97.0,12840.0,99.0,13.0,99.0,32.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.144897e+10,99.0,2.812494e+10,0.0
4,2019-01-05,31346.0,100.0,0.0,97.0,12864.0,99.0,13.0,99.0,32.0,...,28.0,200.0,0.0,100.0,0.0,99.0,3.179422e+10,99.0,2.829816e+10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50313,2019-06-26,8599.0,100.0,0.0,96.0,17168.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201000e+12,99.0,8.356453e+11,0.0
50314,2019-06-27,8599.0,100.0,0.0,96.0,17192.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201312e+12,99.0,8.362658e+11,0.0
50315,2019-06-28,8599.0,100.0,0.0,96.0,17216.0,99.0,21.0,91.0,624.0,...,31.0,200.0,0.0,100.0,0.0,99.0,2.201585e+12,99.0,8.371074e+11,0.0
50316,2019-06-29,8599.0,99.0,2.0,96.0,17240.0,99.0,21.0,91.0,624.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201871e+12,99.0,8.379656e+11,0.0


In [3]:
data[data['label']==1]

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
25159,2019-01-01,32240.0,99.0,2.0,97.0,13752.0,99.0,20.0,99.0,53.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.863063e+11,99.0,1.269325e+11,1.0
25344,2019-01-05,8656.0,99.0,2.0,97.0,13352.0,99.0,16.0,98.0,148.0,...,30.0,200.0,0.0,100.0,0.0,99.0,4.403823e+11,99.0,3.495673e+11,1.0
25527,2019-01-07,38576.0,99.0,2.0,97.0,13290.0,99.0,18.0,93.0,491.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.830638e+12,99.0,5.803942e+11,1.0
25713,2019-01-12,43191.0,100.0,0.0,97.0,14024.0,99.0,22.0,97.0,185.0,...,30.0,199.0,1.0,99.0,8.0,99.0,5.396850e+11,99.0,6.160791e+11,1.0
25894,2019-01-12,28588.0,99.0,2.0,97.0,11450.0,99.0,24.0,99.0,8.0,...,29.0,200.0,0.0,100.0,0.0,99.0,5.544371e+09,99.0,2.655792e+11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49592,2019-06-29,18461.0,99.0,2.0,96.0,17244.0,99.0,26.0,95.0,354.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.014280e+12,99.0,7.793385e+11,1.0
49773,2019-06-29,40280.0,99.0,2.0,96.0,17263.0,99.0,125.0,95.0,308.0,...,30.0,200.0,0.0,100.0,0.0,99.0,9.733655e+11,99.0,7.904370e+11,1.0
49954,2019-06-29,37897.0,99.0,2.0,96.0,17436.0,99.0,18.0,95.0,300.0,...,30.0,200.0,0.0,100.0,0.0,99.0,9.776624e+11,99.0,8.292570e+11,1.0
50135,2019-06-29,28019.0,99.0,2.0,96.0,16638.0,99.0,23.0,100.0,0.0,...,26.0,200.0,0.0,100.0,0.0,99.0,1.471218e+09,99.0,1.148237e+10,1.0


In [5]:
# 라벨이 1인 행을 필터링합니다.
selected_disk_model_1 = data[data['label'] == 1]

# 모델, disk_id, 그리고 날짜 순으로 정렬합니다.
selected_disk_model_1 = selected_disk_model_1.sort_values(by=['disk_id', 'index'])

# disk_id와 model 열만을 선택하여 새로운 데이터 프레임에 저장합니다.
selected_disk_model_1 = selected_disk_model_1[['disk_id']].drop_duplicates().reset_index(drop=True)

# 결과 데이터 프레임을 출력합니다.
selected_disk_model_1.shape

(139, 1)

In [7]:
# disk_ids 데이터프레임에 있는 disk_id와 일치하는 행을 data 데이터프레임에서 불러옴
selected_label_1 = data[data['disk_id'].isin(selected_disk_model_1['disk_id'])]

# ds 컬럼에 따라 정렬
selected_label_1 = selected_label_1.sort_values(by=['disk_id','index'])
# 결과 출력
selected_label_1

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
43078,2019-01-01,531.0,100.0,0.0,97.0,14770.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43079,2019-01-02,531.0,100.0,0.0,97.0,14794.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43080,2019-01-03,531.0,100.0,0.0,97.0,14818.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43081,2019-01-04,531.0,100.0,0.0,97.0,14842.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43082,2019-01-05,531.0,100.0,0.0,97.0,14866.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32756,2019-06-26,44338.0,99.0,2.0,96.0,18504.0,99.0,21.0,99.0,9.0,...,30.0,200.0,0.0,100.0,0.0,99.0,1.395901e+10,99.0,9.413844e+09,0.0
32757,2019-06-27,44338.0,99.0,2.0,96.0,18528.0,99.0,21.0,99.0,9.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.395920e+10,99.0,9.413844e+09,0.0
32758,2019-06-28,44338.0,99.0,2.0,96.0,18552.0,99.0,21.0,99.0,9.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.396014e+10,99.0,9.413844e+09,0.0
32759,2019-06-29,44338.0,99.0,2.0,96.0,18576.0,99.0,21.0,99.0,9.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.396116e+10,99.0,9.413844e+09,0.0


In [9]:
remove_indexes = []
for index, row in selected_label_1.iterrows():
    if row['label'] == 1:
        disk_id = row['disk_id']
        ds = row['index']
        remove_indexes.extend(selected_label_1.index[(selected_label_1['disk_id'] == disk_id) & (selected_label_1['index'] > ds)].tolist())

# 중복된 인덱스를 제거하여 제거할 행들을 구함
remove_indexes = list(set(remove_indexes))

# 제거할 행을 제외한 최종 DataFrame을 생성
del_fail_after_row = selected_label_1.drop(index=remove_indexes)

# 결과 출력
del_fail_after_row

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
43078,2019-01-01,531.0,100.0,0.0,97.0,14770.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43079,2019-01-02,531.0,100.0,0.0,97.0,14794.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43080,2019-01-03,531.0,100.0,0.0,97.0,14818.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43081,2019-01-04,531.0,100.0,0.0,97.0,14842.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43082,2019-01-05,531.0,100.0,0.0,97.0,14866.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32659,2019-03-21,44338.0,99.0,2.0,96.0,16176.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.778025e+09,0.0
32660,2019-03-22,44338.0,99.0,2.0,96.0,16200.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.779657e+09,0.0
32661,2019-03-23,44338.0,99.0,2.0,96.0,16224.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.781085e+09,0.0
32662,2019-03-24,44338.0,99.0,2.0,96.0,16248.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.782419e+09,0.0


In [14]:
del_fail_after_row.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14199 entries, 43078 to 32663
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    14199 non-null  object 
 1   disk_id  14199 non-null  float64
 2   n_5      14199 non-null  float64
 3   r_5      14199 non-null  float64
 4   n_9      14199 non-null  float64
 5   r_9      14199 non-null  float64
 6   n_12     14199 non-null  float64
 7   r_12     14199 non-null  float64
 8   n_177    14199 non-null  float64
 9   r_177    14199 non-null  float64
 10  n_180    14199 non-null  float64
 11  r_180    14199 non-null  float64
 12  n_181    14199 non-null  float64
 13  r_181    14199 non-null  float64
 14  n_182    14199 non-null  float64
 15  r_182    14199 non-null  float64
 16  n_183    14199 non-null  float64
 17  r_183    14199 non-null  float64
 18  n_184    14199 non-null  float64
 19  r_184    14199 non-null  float64
 20  n_187    14199 non-null  float64
 21  r_187   

In [16]:
import pandas as pd

# 'index' 컬럼을 datetime 형태로 변환
del_fail_after_row['index'] = pd.to_datetime(del_fail_after_row['index'])

# 모든 행에 대해 반복
for index, row in del_fail_after_row.iterrows():
    # label 값이 1인 경우
    if row['label'] == 1:
        # 해당 행의 disk_id
        current_disk_id = row['disk_id']
        # 해당 행의 날짜
        current_date = row['index']
        # 30일 이전의 날짜 계산
        days_before = current_date - pd.Timedelta(days=30)
        
        # 같은 disk_id이면서, 날짜가 30일 이내인 행들을 찾아서 label 값을 1로 설정
        del_fail_after_row.loc[(del_fail_after_row['disk_id'] == current_disk_id) &
                             (del_fail_after_row['index'] <= current_date) &
                             (del_fail_after_row['index'] > days_before), 'label'] = 1

del_fail_after_row

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
43078,2019-01-01,531.0,100.0,0.0,97.0,14770.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43079,2019-01-02,531.0,100.0,0.0,97.0,14794.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601918e+11,99.0,1.282129e+12,0.0
43080,2019-01-03,531.0,100.0,0.0,97.0,14818.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43081,2019-01-04,531.0,100.0,0.0,97.0,14842.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
43082,2019-01-05,531.0,100.0,0.0,97.0,14866.0,99.0,20.0,96.0,294.0,...,29.0,200.0,0.0,100.0,0.0,99.0,9.601919e+11,99.0,1.282129e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32659,2019-03-21,44338.0,99.0,2.0,96.0,16176.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.778025e+09,1.0
32660,2019-03-22,44338.0,99.0,2.0,96.0,16200.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.779657e+09,1.0
32661,2019-03-23,44338.0,99.0,2.0,96.0,16224.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.781085e+09,1.0
32662,2019-03-24,44338.0,99.0,2.0,96.0,16248.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.782419e+09,1.0
